# Brusselator model

The dynamics of the oscillating reaction discovered by Belousov and Zhabotinsky, 
can be modeled through the so-called Brusselator model depending on two parameters:

$$
\left\{\begin{aligned}
d_t y_1 & = 1 - (b+1) y_1 + a y_1^2y_2\\
d_t y_2 & = b y_1 - a y_1^2y_2
\end{aligned}\right.
$$

In [3]:
import numpy as np

from scipy.integrate import solve_ivp

from bokeh.io import push_notebook, show, output_notebook
from bokeh.plotting import figure
from bokeh.layouts import column
from bokeh.models import PrintfTickFormatter

from mylib.model import brusselator_model
import mylib.integration as integration

import time

output_notebook(hide_banner=True)

### computational cost

In [ ]:
def plot_rk38_sol():
    
    bm = brusselator_model(a=1, b=3)
    fcn = bm.fcn  
    
    tini = 0. 
    tend = 20.
    
    yini = (1.5, 3)
    
    nt = 1751
    t0 = time.time()
    sol_rk38 = integration.rk38(tini, tend, nt, yini, fcn)
    t1 = time.time()
    
    sol_exa = solve_ivp(fcn, (tini, tend), yini, method="Radau", rtol=1.e-12, atol=1.e-12, t_eval=sol_rk38.t)
    y1_err = np.abs(sol_exa.y[0] - sol_rk38.y[0])
    y2_err = np.abs(sol_exa.y[1] - sol_rk38.y[1])

    fig_sol = figure(x_range=(tini, tend), width=950, height=300, title="Solution")
    fig_sol.x(sol_rk38.t, sol_rk38.y[0], legend="y1", line_width=2)    
    fig_sol.x(sol_rk38.t, sol_rk38.y[1], legend="y2", line_width=2, color="Green")
    fig_sol.legend.location = "top_left"
    
    fig_err = figure(x_range=(tini, tend), y_axis_type="log", plot_height=300, plot_width=950, title="Global error")
    fig_err.yaxis[0].formatter = PrintfTickFormatter(format="%8.1e")
    fig_err.x(sol_rk38.t[1:], y1_err[1:], line_width=2, legend="y1")
    fig_err.x(sol_rk38.t[1:], y2_err[1:], line_width=2, color="Green", legend="y2")
    fig_sol.legend.location = "top_left"

    
    show(column(fig_sol, fig_err))

    print(f"Time to integrate : {(t1-t0):.5f} s")
    print(f"Number of time steps : {(nt-1):d}")
    print(f"Number of function evaluations : {(4*(nt-1)):d}")

plot_rk38_sol()

In [5]:
def plot_radau_cost():

    bm = brusselator_model(a=1, b=3)
    fcn = bm.fcn
    jac = bm.jac
    
    tini = 0. 
    tend = 20.
    
    yini = (1.5, 3)
        
    l_nt = [101, 501, 1001, 5001, 10001, 20001]
    fe_rk38=[]
    norm_rk38=[]
    ffe_rk38=[]
    
    for i in range(0, len(l_nt)):
        sol_rk38 = integration.rk38(tini, tend, l_nt[i], yini, fcn)
        sol_exa = solve_ivp(fcn, (tini, tend), yini, method="Radau", rtol=1.e-12, atol=1.e-12, t_eval=sol_rk38.t)
        err = 0 
        sol_local=sol_rk38
        sol=sol_exa
        fe_rk38.append(l_nt[i]-1)
        ffe_rk38.append(4*(l_nt[i]-1))
        for i in range(sol_local.t.size-1):
            ldt = (sol_local.t[i+1]-sol_local.t[i])/(tend-tini)
            err += ldt * ((sol.y[0,i+1]-sol_local.y[0,i+1])*(sol.y[0,i+1]-sol_local.y[0,i+1]))
        err = np.sqrt(err)    
        norm_rk38.append(err)
    
    
    l_tol = [1.e-2, 1.e-3, 1.e-4, 1.e-5, 1.e-6, 1.e-7, 1.e-9, 1.e-10]
    fe_radau=[]  #number of time step
    norm_radau=[] #global error for y1
    ffe_radau=[]  #evaluation funciton
    fe_dopri5=[]
    norm_dopri5=[]
    ffe_dopri5=[]
    fe_embed=[]
    norm_embed=[]
    ffe_embed=[]
    
    time_radau=[]
    time_dopri5=[]
    time_embed=[]
    
    
    for tol in l_tol:
        time0=time.time()
        sol_rk_emb = integration.rk_embedded(tini, tend, yini, fcn, tol)
        time1=time.time()
        time_embed.append(time1-time0)
        
        time0=time.time()
        sol_dopri5 = solve_ivp(fcn, (tini, tend), yini, method="RK45", rtol=tol, atol=tol)
        time1=time.time()
        time_dopri5.append(time1-time0)
        
        time0=time.time()
        sol_radau = solve_ivp(fcn, (tini, tend), yini, method="Radau", rtol=tol, atol=tol, jac=jac)
        time1=time.time()
        time_radau.append(time1-time0)

        
        
        sol1 = solve_ivp(fcn, (tini, tend), yini, rtol=1.e-12, atol=1.e-12, t_eval=sol_rk_emb.t)
        sol2 = solve_ivp(fcn, (tini, tend), yini, rtol=1.e-12, atol=1.e-12, t_eval=sol_dopri5.t)
        sol3 = solve_ivp(fcn, (tini, tend), yini, rtol=1.e-12, atol=1.e-12, t_eval=sol_radau.t)
        
        ffe_radau.append(sol_radau.nfev)
        ffe_dopri5.append(sol_dopri5.nfev)
        ffe_embed.append(sol_rk_emb.nfev)
        fe_embed.append(sol_rk_emb.t.size-1)
        fe_dopri5.append(sol_dopri5.t.size-1)
        fe_radau.append((sol_radau.t.size-1))
        
        
        
        err = 0 
        sol_local=sol_rk_emb
        sol=sol1
        for i in range(sol_local.t.size-1):
            ldt = (sol_local.t[i+1]-sol_local.t[i])/(tend-tini)
            err += ldt * ((sol.y[0,i+1]-sol_local.y[0,i+1])*(sol.y[0,i+1]-sol_local.y[0,i+1]))
        err = np.sqrt(err)    
        norm_embed.append(err)
        
        err = 0 
        sol_local=sol_dopri5
        sol=sol2
        for i in range(sol_local.t.size-1):
            ldt = (sol_local.t[i+1]-sol_local.t[i])/(tend-tini)
            err += ldt * ((sol.y[0,i+1]-sol_local.y[0,i+1])*(sol.y[0,i+1]-sol_local.y[0,i+1]))
        err = np.sqrt(err)
        norm_dopri5.append(err)
        
        err = 0 
        sol_local=sol_radau
        sol=sol3
        for i in range(sol_local.t.size-1):
            ldt = (sol_local.t[i+1]-sol_local.t[i])/(tend-tini)
            err += ldt * ((sol.y[0,i+1]-sol_local.y[0,i+1])*(sol.y[0,i+1]-sol_local.y[0,i+1]))
        err = np.sqrt(err)  
        norm_radau.append(err)
    
    #time steps in function of accuracy 
    fig = figure(x_axis_type="log", y_axis_type="log", plot_height=500, plot_width=900, \
                 title= "Number of time steps in function of accuracy" )
    fig.x(norm_embed, fe_embed, legend="embedded RK4", color="Indigo")
    fig.line(norm_embed, fe_embed, legend="embedded RK4", color="Indigo")
    fig.x(norm_radau, fe_radau, legend="RADAU", color="Green")
    fig.line(norm_radau, fe_radau, legend="RADAU", color="Green")
    fig.x(norm_dopri5, fe_dopri5, legend="DOPRI5", color="Red")
    fig.line(norm_dopri5, fe_dopri5, legend="DOPRI5", color="Red")
    fig.x(norm_rk38, fe_rk38, legend="RK38", color="Brown")
    fig.line(norm_rk38, fe_rk38, legend="RK38", color="Brown")
    show(fig)
    return 
    fig = figure(x_axis_type="log", y_axis_type="log", plot_height=500, plot_width=900, \
                 title= "Number of time steps in function of tolerance" )
    fig.x(l_tol, fe_embed, legend="embedded RK4", color="Indigo")
    fig.line(l_tol, fe_embed, legend="embedded RK4", color="Indigo")
    fig.x(l_tol, fe_radau, legend="RADAU", color="Green")
    fig.line(l_tol, fe_radau, legend="RADAU", color="Green")
    fig.x(l_tol, fe_dopri5, legend="DOPRI5", color="Red")
    fig.line(l_tol, fe_dopri5, legend="DOPRI5", color="Red")

    show(fig)
    
    
    fig = figure(x_axis_type="log", y_axis_type="log", plot_height=500, plot_width=900, \
                 title= "Number of function evaluations in function of tolerance" )
    fig.x(l_tol, ffe_embed, legend="embedded RK4", color="Indigo")
    fig.line(l_tol, ffe_embed, legend="embedded RK4", color="Indigo")
    fig.x(l_tol, ffe_radau, legend="RADAU", color="Green")
    fig.line(l_tol, ffe_radau, legend="RADAU", color="Green")
    fig.x(l_tol, ffe_dopri5, legend="DOPRI5", color="Red")
    fig.line(l_tol, ffe_dopri5, legend="DOPRI5", color="Red")

    show(fig)
    
    fig = figure(x_axis_type="log", y_axis_type="log", plot_height=500, plot_width=900, \
                 title= "Error of y1 as function of tolerance" )
    fig.x(l_tol, norm_embed, legend="embedded RK4", color="Indigo")
    fig.line(l_tol, norm_embed, legend="embedded RK4", color="Indigo")
    fig.x(l_tol, norm_radau, legend="RADAU", color="Green")
    fig.line(l_tol, norm_radau, legend="RADAU", color="Green")
    fig.x(l_tol, norm_dopri5, legend="DOPRI5", color="Red")
    fig.line(l_tol, norm_dopri5, legend="DOPRI5", color="Red")

    show(fig)
    
    fig = figure(x_axis_type="log", plot_height=500, plot_width=900, \
                 title= "Real time comsumption as function of tolerance" )
    fig.x(l_tol, time_embed, legend="embedded RK4", color="Indigo")
    fig.line(l_tol, time_embed, legend="embedded RK4", color="Indigo")
    fig.x(l_tol, time_radau, legend="RADAU", color="Green")
    fig.line(l_tol, time_radau, legend="RADAU", color="Green")
    fig.x(l_tol, time_dopri5, legend="DOPRI5", color="Red")
    fig.line(l_tol, time_dopri5, legend="DOPRI5", color="Red")

    show(fig)
    
plot_radau_cost()

## Radau5 integration (python implementation)

In [3]:
def plot_radau5_sol():
    
    bm = brusselator_model(a=1, b=3)
    fcn = bm.fcn
    jac = bm.jac
    
    tini = 0. 
    tend = 20.
    
    yini = (1.5, 3)
    
    tol = 1.e-6
    t0 = time.time()
    sol_radau = solve_ivp(fcn, (tini, tend), yini, method="Radau", rtol=tol, atol=tol, jac=jac)
    t1 = time.time()
    
    sol_exa = solve_ivp(fcn, (tini, tend), yini, method="Radau", rtol=1.e-12, atol=1.e-12, t_eval=sol_radau.t, jac=jac)
    y1_err = np.abs(sol_exa.y[0] - sol_radau.y[0])
    y2_err = np.abs(sol_exa.y[1] - sol_radau.y[1])

    fig_sol = figure(x_range=(tini, tend), width=950, height=300, title="Solution")
    fig_sol.x(sol_radau.t, sol_radau.y[0], legend="y1", line_width=2)    
    fig_sol.x(sol_radau.t, sol_radau.y[1], legend="y2", line_width=2, color="Green")
    fig_sol.legend.location = "top_left"
    
    fig_err = figure(x_range=(tini, tend), y_axis_type="log", plot_height=300, plot_width=950, title="Global error")
    fig_err.yaxis[0].formatter = PrintfTickFormatter(format="%8.1e")
    fig_err.x(sol_radau.t[1:], y1_err[1:], line_width=2, legend="y1")
    fig_err.x(sol_radau.t[1:], y2_err[1:], line_width=2, color="Green", legend="y2")
    fig_err.legend.location = "top_left"
    
    show(column(fig_sol, fig_err))
    
    print(f"Time to integrate : {(t1-t0):.5f} s")
    print(f"Number of accepted steps : {(sol_radau.t.size-1):d}")
    print(f"Number of function evaluations : {sol_radau.nfev:d}")
    print(f"Number of evaluations of the jacobian : {sol_radau.njev:d}")
    print(f"Number of LU decompositions {sol_radau.nlu:d}")
    
plot_radau5_sol()

Time to integrate : 0.15626 s
Number of accepted steps : 278
Number of function evaluations : 2206
Number of evaluations of the jacobian : 51
Number of LU decompositions 228


* 因为是implicit，所以需
* 并不是每次都重新算jacobian
* 最后两个不是相等的，因为减少jacobian计算，“迫不得已”再计算下一次的

## Runge-Kutta "3/8 method"

In [7]:
def plot_rk38_sol():
    
    bm = brusselator_model(a=1, b=3)
    fcn = bm.fcn  
    
    tini = 0. 
    tend = 20.
    
    yini = (1.5, 3)
    
    nt = 1751
    t0 = time.time()
    sol_rk38 = integration.rk38(tini, tend, nt, yini, fcn)
    t1 = time.time()
    
    sol_exa = solve_ivp(fcn, (tini, tend), yini, method="Radau", rtol=1.e-12, atol=1.e-12, t_eval=sol_rk38.t)
    y1_err = np.abs(sol_exa.y[0] - sol_rk38.y[0])
    y2_err = np.abs(sol_exa.y[1] - sol_rk38.y[1])

    fig_sol = figure(x_range=(tini, tend), width=950, height=300, title="Solution")
    fig_sol.x(sol_rk38.t, sol_rk38.y[0], legend="y1", line_width=2)    
    fig_sol.x(sol_rk38.t, sol_rk38.y[1], legend="y2", line_width=2, color="Green")
    fig_sol.legend.location = "top_left"
    
    fig_err = figure(x_range=(tini, tend), y_axis_type="log", plot_height=300, plot_width=950, title="Global error")
    fig_err.yaxis[0].formatter = PrintfTickFormatter(format="%8.1e")
    fig_err.x(sol_rk38.t[1:], y1_err[1:], line_width=2, legend="y1")
    fig_err.x(sol_rk38.t[1:], y2_err[1:], line_width=2, color="Green", legend="y2")
    fig_sol.legend.location = "top_left"

    
    show(column(fig_sol, fig_err))

    print(f"Time to integrate : {(t1-t0):.5f} s")
    print(f"Number of time steps : {(nt-1):d}")
    print(f"Number of function evaluations : {(4*(nt-1)):d}")

plot_rk38_sol()

Time to integrate : 0.07815 s
Number of time steps : 1750
Number of function evaluations : 7000


* 上面两种的情况误差差不多，
* 但是pas更多了，尽管我们有更多的evaluation但是我们在时间上还是更快乐
* pas fixed

## Embedded method

In [4]:
def plot_rk_emb_sol():
    
    bm = brusselator_model(a=1, b=3)
    fcn = bm.fcn  
    
    tini = 0. 
    tend = 20.
    
    yini = (1.5, 3)
    
    tol = 2.5e-8
    tol = 1.e-6
    t0 = time.time()
    sol_rk_emb = integration.rk_embedded(tini, tend, yini, fcn, tol)
    t1 = time.time()

    sol_exa = solve_ivp(fcn, (tini, tend), yini, method="Radau", rtol=1.e-12, atol=1.e-12, t_eval=sol_rk_emb.t)
    y1_err = np.abs(sol_exa.y[0] - sol_rk_emb.y[0])
    y2_err = np.abs(sol_exa.y[1] - sol_rk_emb.y[1])

    fig_sol = figure(x_range=(tini, tend), width=950, height=300, title="Solution")
    fig_sol.x(sol_rk_emb.t, sol_rk_emb.y[0], legend="y1", line_width=2)    
    fig_sol.x(sol_rk_emb.t, sol_rk_emb.y[1], legend="y2", line_width=2, color="Green")
    fig_sol.legend.location = "top_left"
    
    fig_err = figure(x_range=(tini, tend), y_axis_type="log", plot_height=300, plot_width=950, title="Global error")
    fig_err.yaxis[0].formatter = PrintfTickFormatter(format="%8.1e")
    fig_err.x(sol_rk_emb.t[1:], y1_err[1:], line_width=2, legend="y1")
    fig_err.x(sol_rk_emb.t[1:], y2_err[1:], line_width=2, color="Green", legend="y2")
    fig_sol.legend.location = "top_left"
    
    show(column(fig_sol, fig_err))

    print(f"Time to integrate : {(t1-t0):.5f} s")
    print(f"Number of accepted steps : {(sol_rk_emb.t.size-1):d}")
    print(f"Number of function evaluations : {sol_rk_emb.nfev:d}")

plot_rk_emb_sol()

Time to integrate : 0.03125 s
Number of accepted steps : 276
Number of function evaluations : 1233


* 用三阶来计算四届误差，从而优化步长
* 因为并不是和evaluation完全一致，因为计算三阶也需要时间

## Dopri5 method

In [3]:
def plot_dopri5_sol():
    
    bm = brusselator_model(a=1, b=3)
    fcn = bm.fcn
    jac = bm.jac
    
    tini = 0. 
    tend = 20.
    
    yini = (1.5, 3)
    
    tol = 5.e-9
    t0 = time.time()
    sol_dopri5 = solve_ivp(fcn, (tini, tend), yini, method="RK45", rtol=tol, atol=tol)
    t1 = time.time()
    
    sol_exa = solve_ivp(fcn, (tini, tend), yini, method="RK45", rtol=1.e-12, atol=1.e-12, t_eval=sol_dopri5.t)
    y1_err = np.abs(sol_exa.y[0] - sol_dopri5.y[0])
    y2_err = np.abs(sol_exa.y[1] - sol_dopri5.y[1])

    fig_sol = figure(x_range=(tini, tend), width=950, height=300, title="Solution")
    fig_sol.x(sol_dopri5.t, sol_dopri5.y[0], legend="y1", line_width=2)    
    fig_sol.x(sol_dopri5.t, sol_dopri5.y[1], legend="y2", line_width=2, color="Green")
    fig_sol.legend.location = "top_left"
    
    fig_err = figure(x_range=(tini, tend), y_axis_type="log", plot_height=300, plot_width=950, title="Global error")
    fig_err.yaxis[0].formatter = PrintfTickFormatter(format="%8.1e")
    fig_err.x(sol_dopri5.t[1:], y1_err[1:], line_width=2, legend="y1")
    fig_err.x(sol_dopri5.t[1:], y2_err[1:], line_width=2, color="Green", legend="y2")
    fig_sol.legend.location = "top_left"

    show(column(fig_sol, fig_err))
    
    print(f"Time to integrate : {(t1-t0):.5f} s")
    print(f"Number of accepted steps : {(sol_dopri5.t.size-1):d}")
    print(f"Number of function evaluations : {sol_dopri5.nfev:d}")
    
plot_dopri5_sol()

Time to integrate : 0.04350 s
Number of accepted steps : 300
Number of function evaluations : 2102


* 步数更少，evaluation更少，因为order上去了

## Radau method (fortran implementation)

In [4]:
def plot_radau5_fortran_sol():
    
    bm = brusselator_model(a=1, b=3)
    fcn = bm.fcn
    fcn_radau = bm.fcn_radau
    jac = bm.jac
    
    tini = 0. 
    tend = 20.
    
    yini = (1.5, 3)
    
    tol = 5.e-8
    t0 = time.time()
    sol_radau = integration.radau5(tini, tend, yini, fcn_radau, njac=2, rtol=tol, atol=tol, iout=1)
    t1 = time.time()
    
    sol_exa = solve_ivp(fcn, (tini, tend), yini, method="Radau", rtol=1.e-12, atol=1.e-12, t_eval=sol_radau.t, jac=jac)
    y1_err = np.abs(sol_exa.y[0] - sol_radau.y[0])
    y2_err = np.abs(sol_exa.y[1] - sol_radau.y[1])

    fig_sol = figure(x_range=(tini, tend), width=950, height=300, title="Solution")
    fig_sol.x(sol_radau.t, sol_radau.y[0], legend="y1", line_width=2)    
    fig_sol.x(sol_radau.t, sol_radau.y[1], legend="y2", line_width=2, color="Green")
    fig_sol.legend.location = "top_left"
    
    fig_err = figure(x_range=(tini, tend), y_axis_type="log", plot_height=300, plot_width=950, title="Global error")
    fig_err.yaxis[0].formatter = PrintfTickFormatter(format="%8.1e")
    fig_err.x(sol_radau.t[1:], y1_err[1:], line_width=2, legend="y1")
    fig_err.x(sol_radau.t[1:], y2_err[1:], line_width=2, color="Green", legend="y2")
    fig_err.legend.location = "top_left"
    
    show(column(fig_sol, fig_err))
    
    print(f"Time to integrate : {(t1-t0):.5f} s")
    print(f"Number of function evaluations : {sol_radau.nfev:d}")
    print(f"Number of jacobian evaluations : {sol_radau.njev:d}")
    print(f"Number of computed steps : {sol_radau.nstep:d}")
    print(f"Number of accepted steps : {sol_radau.naccpt:d}")
    print(f"Number of rejected steps : {sol_radau.nrejct:d}")
    print(f"Number of LU decompositions : {sol_radau.ndec:d}")
    print(f"Number of forward-backward substitutions : {sol_radau.nsol:d}")    
    
plot_radau5_fortran_sol()

OSError: [WinError 126] 找不到指定的模块。

* tolerance大概在-6级别，时间快很多，function evaluation 1821
* number of jacobian evaluation 214
* 和python出来的结果不一致，也是因为不同的写法，再加上fortran语言本身就更快

* 有不同的方法，问题是，知道raideur，到底该用那种方法
* 事实上当system不是很恐怖的时候，完全也可以使用explicit的高阶
* CR中要写出理由